# Bulding an AI Agent

This notebook will walk you through the process of building an AI agent. According to Google, an **AI agent** is "a software system that uses AI to pursue goals and complete tasks on behalf of users. It demonstrates reasoning, planning, and memory, and possesses a level of autonomy to make decisions, learn, and adapt."

In this notebook, we explore the creation of an **AI agent** using the [LangChain](https://www.langchain.com/) framework.

### Install the neccessary libraries.

In [1]:
%pip install -qU "langchain[groq]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -qU langchain-community langgraph tavily-python langgraph-checkpoint-sqlite

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -qU "langchain[groq]"

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from dotenv import load_dotenv

### Load the environment variables required for this project.

In [6]:
# Load the environment variables from the .env file.
load_dotenv()

# Access the environment variables.
groq_api_key = os.getenv("GROQ_API_KEY")
langsmith_tracing = os.getenv("LANGSMITH_TRACING")
langsmith_endpoint = os.getenv("LANGSMITH_ENDPOINT")
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
langsmith_project = os.getenv("LANGSMITH_PROJECT")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# # Print the variables to verify.
# print("GROQ API Key:", groq_api_key)
# print("LangSmith Tracing:", langsmith_tracing)
# print("LangSmith Endpoint:", langsmith_endpoint)
# print("LangSmith API Key:", langsmith_api_key)
# print("LangSmith Project:", langsmith_project)

In [7]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="llama-3.3-70b-versatile", model_provider="groq")

from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="Hello, tell me about GENAI")])

In [8]:
print(response.content)

GENAI is an AI model developed to process and generate human-like language. GENAI stands for Generalized AI, and it's designed to be a versatile and adaptable language model.

Here are some key features of GENAI:

1. **Language Understanding**: GENAI is trained on a massive dataset of text from various sources, allowing it to understand the nuances of language, including grammar, syntax, and context.
2. **Text Generation**: GENAI can generate high-quality text based on a given prompt or topic. It can create articles, stories, emails, and even entire books.
3. **Conversational Interface**: GENAI can engage in natural-sounding conversations, using context and understanding to respond to questions and statements.
4. **Knowledge Retrieval**: GENAI has been trained on a vast amount of text data, allowing it to retrieve and provide information on a wide range of topics.
5. **Continuous Learning**: GENAI is designed to learn and improve over time, adapting to new data and user interactions.



### We will be using Tavily (a search engine), a tool for searching the web.

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=3)
search_results = search.invoke("Who's the best footballer in the world?")

print(search_results)

tools = [search]

[{'title': 'Best football players in the world ranked 2025 - Radio Times', 'url': 'https://www.radiotimes.com/tv/sport/football/best-football-players-world/', 'content': 'He has started to move through the gears for Real Madrid and by the end of the campaign, he will have netted a ferocious haul of goals and, in all likelihood, trophies.\nWho is the best football player in the world?\nLionel Messi has been the undisputed king of the football world for the past decade, but his time at the top has drawn to a close. Kylian Mbappe is our best football player in the world 2025. [...] Who is the best player in world football? For the best part of two decades, the answer has been a coin toss. No longer.\nAd\nLionel Messi and Cristiano Ronaldo were the two greatest footballers of a generation, the former earning the Greatest of All Time tag in many quarters after capping off an outrageous career with the World Cup, the ultimate prize. [...] 2. Mohamed Salah (Liverpool)', 'score': 0.8993429}, {

Using the `.bind_tools()` method to give the language model knowledge of the Tavily tool.

In [10]:
model_with_tools = model.bind_tools(tools)

response = model_with_tools.invoke([HumanMessage(content="what's computer networking?")])

print(response.content)

### Create and run the agent.

In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model=model, tools=tools)

response  = agent_executor.invoke(
    {"messages": [HumanMessage(content="Who's the best footballer in the world?")]}
    )

print(response["messages"])

[HumanMessage(content="Who's the best footballer in the world?", additional_kwargs={}, response_metadata={}, id='04ff4e80-de86-4db8-b521-15ea773b7872'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ndjz', 'function': {'arguments': '{"query": "best footballer in the world"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 288, 'total_tokens': 311, 'completion_time': 0.083636364, 'prompt_time': 0.0182384, 'queue_time': 0.101058303, 'total_time': 0.101874764}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e432eeb4-1d89-48d3-b029-056a2b54d7e6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'best footballer in the world'}, 'id': 'call_ndjz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 288, 'output_tokens': 23, 'total_tokens': 311}), ToolMessage(conte

### Stream the agent's response.

In [12]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="Who's the best footballer in the world?")]},
    stream_mode="values"
    ):
    print(step["messages"][-1].pretty_print())

================================ Human Message =================================

Who's the best footballer in the world?
None
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_xd76)
 Call ID: call_xd76
  Args:
    query: best footballer in the world
None
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Best football players in the world ranked 2025 - Radio Times", "url": "https://www.radiotimes.com/tv/sport/football/best-football-players-world/", "content": "He has started to move through the gears for Real Madrid and by the end of the campaign, he will have netted a ferocious haul of goals and, in all likelihood, trophies.\nWho is the best football player in the world?\nLionel Messi has been the undisputed king of the football world for the past decade, but his time at the top has drawn to a close. Kylian Mbappe is our best footba

### Adding a memory

The agent is stateless because it does not retain information from prior interactions, thereby preventing it from recalling previous conversations.

In [13]:
from langgraph.checkpoint.memory import MemorySaver

# Create a memory saver instance.
memory = MemorySaver()

agent_executor = create_react_agent(model=model, tools=tools, checkpointer=memory)

config = {"configurable": {"thread_id": 66}}


In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im osunba silas!")]}, config
):
    print(chunk)
    print("-----")

{'agent': {'messages': [AIMessage(content="It's nice to meet you, Osunba Silas. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 286, 'total_tokens': 315, 'completion_time': 0.105454545, 'prompt_time': 0.025766565, 'queue_time': 0.100645991, 'total_time': 0.13122111}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'stop', 'logprobs': None}, id='run--1924e980-77e9-4adb-bb8f-63d4b98e57f2-0', usage_metadata={'input_tokens': 286, 'output_tokens': 29, 'total_tokens': 315})]}}
-----


In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="yes, do you know who i am? what's my name?")]}, config
):
    print(chunk)
    print("-----")

{'agent': {'messages': [AIMessage(content="You've already told me your name, which is Osunba Silas.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 338, 'total_tokens': 355, 'completion_time': 0.061818182, 'prompt_time': 0.029752245, 'queue_time': 0.101474453, 'total_time': 0.091570427}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'stop', 'logprobs': None}, id='run--389ce976-77f4-4cdc-80e4-daa7f0f5f156-0', usage_metadata={'input_tokens': 338, 'output_tokens': 17, 'total_tokens': 355})]}}
-----
